In [1]:
# Indusries
# Job Categories
# Locations ==> Suburb (optional)
# Job Title ==> Do (10) categories (cleaning) ==> Count Vectorizer
# Work Type
# Dates <== missing
# Job Description

# =============================================
#### Dummies
# =============================================
# ==> Industries
# ==> Location
# ==> Job Title

# =============================================
#### CountVectorizer
# =============================================
# ==> Job Title ==> Experiences columns (Senior, Junior, Entry-level)


# Step 1. Do Linear Regression

In [2]:
####
# Step 0: graph salary distribution (neg, pos skew)
# ==> EDA:
#     Contract DS (140 - 200k)
#     Contract DA (45 - 60k)
#     Contract (25%) | Fulltime (75%)
#     Industries (20-30) ==> to (10-15)
#     Job Titles ==> (DS [Junior, Senior/Manager, just DS], 
#                     DA [Junior, Senior/Manager, just DA]
#                     DE [Junior, Senior/Manager, just DE])
#                 ==> to (12-15)
#     Categories (200) ==> to (20)

# Step 1: Regression Problems
# ==> issues: 
#     not good model
# ==> treat as classification problem


In [3]:
import pandas as pd
import numpy as np
import re

# Folder
target_file = './training/training_next_16052019.csv'

In [4]:
frame = pd.read_csv(target_file, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Over the last 2 years this business has been s...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Scientist Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/38892320?type=stan...,87575
1,The Data Architect is responsible for enterpri...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Architect Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/39015289?type=stan...,114181
2,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935314?type=stan...,87747
3,The Data Engineer (Cloud/Data Lake) will play ...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Engineer - Cloud / Data Lake Job in Brisb...,Full Time,https://www.seek.com.au/job/39015517?type=stan...,114181
4,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935330?type=stan...,87747


In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 7 columns):
content       1261 non-null object
industries    1261 non-null object
location      1261 non-null object
title         1261 non-null object
type_empl     1261 non-null object
url           1261 non-null object
wage          1261 non-null int64
dtypes: int64(1), object(6)
memory usage: 69.0+ KB


In [6]:
frame['title'].value_counts()

Senior Business Analyst Job in Sydney - SEEK                                                   10
Business Analyst Job in Sydney - SEEK                                                          10
Financial Analyst Job in Sydney - SEEK                                                          7
Data Analyst Job in Sydney - SEEK                                                               7
Campaign Project Officer Job in Sydney - SEEK                                                   5
Test Analyst Job in Brisbane - SEEK                                                             5
Senior Commercial Analyst Job in Sydney - SEEK                                                  5
Data Analyst Job in Melbourne - SEEK                                                            5
Data Scientist Job in Melbourne - SEEK                                                          4
Software Engineer Job in Sydney - SEEK                                                          4
DevOps Engineer - AW

In [7]:
# Split Suburb out of content
def get_main_location(regex, sample):
#     print("In:", sample)
    result = re.search(regex, str(sample))

    if result:
#         print(result[0])
        result = result[0]
        result = result.replace(",","").lstrip().rstrip()
        return result
    elif len(str(sample)) > 3:
        return str(sample)
    return ''

# 
def get_suburb(regex, sample):
    result = re.sub(regex,"",str(sample))
    
    return result

In [8]:
frame['suburb'] = frame['location'].map(lambda row: get_suburb(r"(^[A-Za-z]+, )",row))
frame['location'] = frame['location'].map(lambda row: get_main_location(r"(^[A-Za-z]+, )",row))
frame['title'] = frame['title'].map(lambda row: row[:(row.find("Job")-1)])

In [9]:
frame['title'].value_counts()

Data Analyst                                                                  15
Business Analyst                                                              14
Senior Business Analyst                                                       13
Financial Analyst                                                              8
Test Analyst                                                                   8
Postdoctoral Research Fellow                                                   7
Data Scientist                                                                 7
Senior Data Scientist                                                          6
Commercial Analyst                                                             5
Campaign Project Officer                                                       5
Senior Commercial Analyst                                                      5
Campaign Analyst                                                               5
Data Engineer               

In [10]:
fields = ['Data Analyst','Data Science','Data Scientist']
exp_levels = ['Senior','General','Junior']
title_mapping_dictionary = {
    exp_levels[0] : ['Senior','Lead','Head','Manager','Director','Snr','Sr','sr','Supervisor','Master'],
    exp_levels[1] : ['all','All','Experienced'],
    exp_levels[2] : ['Jr','Junior','jr','Tableau']
}
context_mapping_dictionary = {
    fields[0]: ['Data Analytics','Data Analyst','Data & Analytics','Big Data','Data Specialist',
                'Data Business Analyst','Data Visualisation','Financial Analyst','Finance Analyst',
                'Reporting Analyst','Data Modeler','Data and Analytics','Python Developer',
                'Project Manager - Data','Data Migration Specialist'],
    fields[1]: ['Data Science','Machine Learning','modelling','DATA SCIENCE','Data Modeller'],
    fields[2]: ['AI','ai','Deep Learning','Data Scientists','Data Scientist'],
    'Engineers': ['Software Engineer','DevOps Engineer - AWS','DevOps Engineer','Java Developer','Data Engineer',
                  'Infrastructure Engineer','Full Stack Developer','Front End Developer','SYSTEMS ENGINEER',
                  'Full Stack PHP Developer','iOS Developer','SQL Developer','BI Developer','Software Developer',
                  'GIS Developer','Microstrategy Developer','JavaScript Developer','Datastage Developer'],
    'Analyst': ['Analyst','business analyst','Business Analyst','Test Analyst','Commercial Analyst',
                'Technical Business Analyst','Marketing Analyst','Marketing Analytics','Analytics'],
    'Academic': ['Lecturer','Postdoctoral','Research Assistant','Lecturer','ACADEMIC','LECTURER','RESEARCH',
                 'Research'],
    'Finance': ['Finance','Financial'],
    'Account': ['Account','Account Manager','Product Manager']
}     

In [11]:
# Store all "keysearch" values in separate dataFrame.
# frame1 = []
# for value in fields:
#     result = frame[frame['title'] == value]
#     frame1.append(result)
# df = pd.concat(frame1)
# out_index = df.index.values
# df = df.reset_index(drop=True)
# df.head()
# print(df.shape)
# print("==============\n")
# frame.drop(out_index,inplace=True)
# frame.info()

In [12]:
# Round2: title include "keysearch" plus
# New dataFrame columns store the rest.
# titles = frame['title'].values
# new_titles = [''] * len(titles)
# title_left_over = [''] * len(titles)

# for i in fields:
#     for key, title in enumerate(titles):
#         if i in title:
#             # Push left over text to list
#             left_over = title.replace(i,"")
#             left_over = re.sub(r"([^a-zA-z\s])"," ",left_over)
#             left_over = re.sub(r"\s+"," ",left_over)

#             title_left_over[key] = left_over.lstrip().rstrip()
            
#             # Granted title as expected field
#             new_titles[key] = i
#         else:
#             title_left_over[key] = ''    
#             new_titles[key] = title
# df_title_lover = pd.DataFrame(title_left_over)
# frame['title'] = new_titles
# frame['left_over_title'] = title_left_over

In [13]:
titles = frame['title'].values
new_titles = frame['title'].values
experiences = [''] * len(titles)
title_left_over = [''] * len(titles)

for key, value in enumerate(new_titles):
    print("value: ", value)
    is_not_simple = True
    # Simple mapping level
    for exp in exp_levels:
        if exp in value:       
            print("Exp in Exp_levels : ", exp,' ------- ' ,value)
            experiences[key] = exp
            title_left_over[key] = value
            new_titles[key] = value.replace(exp,"").rstrip().lstrip()            
            is_not_simple = False
            print("New title: ", new_titles[key])
            break
    
    # Dictionary mapping level
    if is_not_simple:
        for k_dict, v_dict in title_mapping_dictionary.items():
            for v_dict_item in v_dict:
                if v_dict_item in value:
                    print("Exp mapping dict : ", k_dict,' ------- ' ,value)
                    experiences[key] = k_dict
                    title_left_over[key] = value
                    new_titles[key] = value.replace(v_dict_item,"").rstrip().lstrip()
                    new_titles[key] = new_titles[key].replace(k_dict,"").rstrip().lstrip()
                    print("New title: ", new_titles[key])
                    break
                else:
                    experiences[key] = 'General'
    
    # Context mapping level as dictionary
    # For updating title only, NOT experiences
    for k_dict, v_dict in context_mapping_dictionary.items():
        for v_dict_item in v_dict:
            if v_dict_item in value:
                print("[Title] mapping dict : ", k_dict,' ------- ' ,value)
                title_left_over[key] = value
#                 title_left_over[key] = value.replace(v_dict_item,".__.")
                new_titles[key] = k_dict
                break
        
    print("=================\n")
frame['title'] = new_titles
frame['experiences'] = experiences
frame['title_left_over'] = title_left_over

value:  Data Scientist
[Title] mapping dict :  Data Scientist  -------  Data Scientist

value:  Data Architect

value:  Applications Developer Enterprise Data Warehouse (EDW)

value:  Data Engineer - Cloud / Data Lake
[Title] mapping dict :  Engineers  -------  Data Engineer - Cloud / Data Lake

value:  Applications Developer Enterprise Data Warehouse (EDW)

value:  Senior Data Scientist
Exp in Exp_levels :  Senior  -------  Senior Data Scientist
New title:  Data Scientist
[Title] mapping dict :  Data Scientist  -------  Senior Data Scientist

value:  Director of Big Data
Exp mapping dict :  Senior  -------  Director of Big Data
New title:  of Big Data
[Title] mapping dict :  Data Analyst  -------  Director of Big Data

value:  Research Scientist In Field Robotics
[Title] mapping dict :  Academic  -------  Research Scientist In Field Robotics

value:  Senior Data Scientist
Exp in Exp_levels :  Senior  -------  Senior Data Scientist
New title:  Data Scientist
[Title] mapping dict :  Dat

value:  Reporting Analyst
[Title] mapping dict :  Data Analyst  -------  Reporting Analyst
[Title] mapping dict :  Analyst  -------  Reporting Analyst

value:  Lead Analyst - Insurance Analytics and Reporting
Exp mapping dict :  Senior  -------  Lead Analyst - Insurance Analytics and Reporting
New title:  Analyst - Insurance Analytics and Reporting
[Title] mapping dict :  Analyst  -------  Lead Analyst - Insurance Analytics and Reporting

value:  Research Scientist - Data Integration
[Title] mapping dict :  Academic  -------  Research Scientist - Data Integration

value:  Data and Reporting Analyst
[Title] mapping dict :  Data Analyst  -------  Data and Reporting Analyst
[Title] mapping dict :  Analyst  -------  Data and Reporting Analyst

value:  Data Analyst
[Title] mapping dict :  Data Analyst  -------  Data Analyst
[Title] mapping dict :  Analyst  -------  Data Analyst

value:  Data Analyst
[Title] mapping dict :  Data Analyst  -------  Data Analyst
[Title] mapping dict :  Analyst 

Exp mapping dict :  Senior  -------  Commercial Finance Manager
New title:  Commercial Finance
[Title] mapping dict :  Finance  -------  Commercial Finance Manager

value:  Senior Java Developer well versed in DevOps 
Exp in Exp_levels :  Senior  -------  Senior Java Developer well versed in DevOps 
New title:  Java Developer well versed in DevOps
[Title] mapping dict :  Engineers  -------  Senior Java Developer well versed in DevOps 

value:  Operations Analyst | Energy Start-Up| $80-100k+Super
[Title] mapping dict :  Analyst  -------  Operations Analyst | Energy Start-Up| $80-100k+Super

value:  National Account Manager - Coles 
Exp mapping dict :  Senior  -------  National Account Manager - Coles 
New title:  National Account  - Coles
[Title] mapping dict :  Account  -------  National Account Manager - Coles 

value:  Marketing Cloud Business Analyst
[Title] mapping dict :  Analyst  -------  Marketing Cloud Business Analyst

value:  Security Analyst / SIEM/SIEM Engineer
[Title] mapp


value:  Automation Tester

value:  Commercial Accountant - Operations
[Title] mapping dict :  Account  -------  Commercial Accountant - Operations

value:  Fees Reporting Analyst
[Title] mapping dict :  Data Analyst  -------  Fees Reporting Analyst
[Title] mapping dict :  Analyst  -------  Fees Reporting Analyst

value:  Applications Support Analyst - SQL Experience Required
[Title] mapping dict :  Analyst  -------  Applications Support Analyst - SQL Experience Required

value:  Transition Specialist (Student Services)

value:  Transition Specialist (Student Services)

value:  Technical Business Analyst
[Title] mapping dict :  Analyst  -------  Technical Business Analyst

value:  Junior Business Analyst, Digital
Exp in Exp_levels :  Junior  -------  Junior Business Analyst, Digital
New title:  Business Analyst, Digital
[Title] mapping dict :  Analyst  -------  Junior Business Analyst, Digital

value:  Technical Support Representative

value:  Payroll Analyst
[Title] mapping dict :  An


value:  Data Scientist (Contract)
[Title] mapping dict :  Data Scientist  -------  Data Scientist (Contract)

value:  Research Data Scientist
[Title] mapping dict :  Data Scientist  -------  Research Data Scientist
[Title] mapping dict :  Academic  -------  Research Data Scientist

value:  Analytics Manager
Exp mapping dict :  Senior  -------  Analytics Manager
New title:  Analytics
[Title] mapping dict :  Analyst  -------  Analytics Manager

value:  Senior Tableau Data Visualisation Consultant - 3 Month Contract
Exp in Exp_levels :  Senior  -------  Senior Tableau Data Visualisation Consultant - 3 Month Contract
New title:  Tableau Data Visualisation Consultant - 3 Month Contract
[Title] mapping dict :  Data Analyst  -------  Senior Tableau Data Visualisation Consultant - 3 Month Contract

value:  Head of Insights & Analytics - Digital Media
Exp mapping dict :  Senior  -------  Head of Insights & Analytics - Digital Media
New title:  of Insights & Analytics - Digital Media
[Title] ma

[Title] mapping dict :  Account  -------  Senior Product Manager

value:  Solutions Engineer / Cloud App Support - AWESOME TEAM - $70-100K+ Super+Perks

value:  PROJECT COORDINATOR - ACADEMIC INTEGRITY
[Title] mapping dict :  Academic  -------  PROJECT COORDINATOR - ACADEMIC INTEGRITY

value:  Project Manager - NSW Health 
Exp mapping dict :  Senior  -------  Project Manager - NSW Health 
New title:  Project  - NSW Health

value:  Sr APJ Marketing Analytics Manager
Exp mapping dict :  Senior  -------  Sr APJ Marketing Analytics Manager
New title:  Sr APJ Marketing Analytics
[Title] mapping dict :  Analyst  -------  Sr APJ Marketing Analytics Manager

value:  Sr APJ Marketing Analytics Manager
Exp mapping dict :  Senior  -------  Sr APJ Marketing Analytics Manager
New title:  Sr APJ Marketing Analytics
[Title] mapping dict :  Analyst  -------  Sr APJ Marketing Analytics Manager

value:  Sr APJ Marketing Analytics Manager
Exp mapping dict :  Senior  -------  Sr APJ Marketing Analytics Ma

value:  Business Development Manager - Enterprise - SaaS
Exp mapping dict :  Senior  -------  Business Development Manager - Enterprise - SaaS
New title:  Business Development  - Enterprise - SaaS

value:  Rostering Allocation Coordinator
Exp mapping dict :  General  -------  Rostering Allocation Coordinator
New title:  Rostering ocation Coordinator

value:  National Account Executive - Sydney
[Title] mapping dict :  Account  -------  National Account Executive - Sydney

value:  Senior business analyst
Exp in Exp_levels :  Senior  -------  Senior business analyst
New title:  business analyst
[Title] mapping dict :  Analyst  -------  Senior business analyst

value:  Digital Content Specialist

value:  Marketing and Client Service Lead
Exp mapping dict :  Senior  -------  Marketing and Client Service Lead
New title:  Marketing and Client Service

value:  Sports Data Quality Assurance Manager - Oceania
Exp mapping dict :  Senior  -------  Sports Data Quality Assurance Manager - Oceania
Ne

Exp in Exp_levels :  Junior  -------  Junior Business Analyst
New title:  Business Analyst
[Title] mapping dict :  Analyst  -------  Junior Business Analyst

value:  SEO Specialist

value:  Analyst, Fund & Investment Operations
[Title] mapping dict :  Analyst  -------  Analyst, Fund & Investment Operations

value:  Forensic Accounting Principal
[Title] mapping dict :  Account  -------  Forensic Accounting Principal

value:  Analyst
[Title] mapping dict :  Analyst  -------  Analyst

value:  SQL Developer/Admin
[Title] mapping dict :  Engineers  -------  SQL Developer/Admin

value:  Commercial Finance Analyst
[Title] mapping dict :  Data Analyst  -------  Commercial Finance Analyst
[Title] mapping dict :  Analyst  -------  Commercial Finance Analyst
[Title] mapping dict :  Finance  -------  Commercial Finance Analyst

value:  Software Engineer (NodeJS, React, AWS)
[Title] mapping dict :  Engineers  -------  Software Engineer (NodeJS, React, AWS)

value:  Business Analyst - Finance and Le

New title:  Marketing  - Private Heath Insurance and Insurance

value:  SAP Technical Lead - S4HANA
Exp mapping dict :  Senior  -------  SAP Technical Lead - S4HANA
New title:  SAP Technical  - S4HANA

value:  Payroll and Employee Service Officers NSW State Government

value:  Medical Laboratory Scientist

value:  Sales / Office Administrator

value:  Lab Analysis / QC - Complementary Medicine Manufacturing

value:  Consultant, HRIS Support

value:  Consultant, HRIS Support

value:  Procurement Specialist  - NSW State Government - Parramatta

value:  Medical Laboratory Assistant - Pathology Collector

value:  Corporate Receptionist - Guest Services 

value:  Customer Service Officer 

value:  Risk Co-Ordinator - NDIS Provider 

value:  Business Analyst - Wyong - 3m + extensions
[Title] mapping dict :  Analyst  -------  Business Analyst - Wyong - 3m + extensions

value:  Financial Analyst
[Title] mapping dict :  Data Analyst  -------  Financial Analyst
[Title] mapping dict :  Analyst  -


value:  Research Assistant
[Title] mapping dict :  Academic  -------  Research Assistant

value:  Postdoctoral Research Fellow
[Title] mapping dict :  Academic  -------  Postdoctoral Research Fellow

value:  Road Safety Projects Officer

value:  Full Stack Developer
[Title] mapping dict :  Engineers  -------  Full Stack Developer

value:  Medical Advisor

value:  Sales assistant

value:  Laser Scanner-CAD Technician-Modeller

value:  Senior Environmental Consultant - Contaminated Land Management
Exp in Exp_levels :  Senior  -------  Senior Environmental Consultant - Contaminated Land Management
New title:  Environmental Consultant - Contaminated Land Management

value:  IT Manager - Professional Services
Exp mapping dict :  Senior  -------  IT Manager - Professional Services
New title:  IT  - Professional Services

value:  Java Developer - Support Engineer
[Title] mapping dict :  Engineers  -------  Java Developer - Support Engineer

value:  Postdoctoral Researcher – Person-centred He

In [14]:
# frame['experiences'].value_counts()

In [21]:
frame['title'].value_counts()

Analyst                                                                      377
Academic                                                                     123
Engineers                                                                     88
Account                                                                       79
Finance                                                                       61
Data Scientist                                                                56
Data Analyst                                                                  15
Data Science                                                                   6
Digital Marketing                                                              4
Advisor Human Resources                                                        4
Project Delivery                                                               4
Project                                                                        3
Solution Architect          

In [16]:
frame['title_left_over'].value_counts()

                                                                        261
Data Analyst                                                             15
Business Analyst                                                         14
Senior Business Analyst                                                  13
Test Analyst                                                              8
Financial Analyst                                                         8
Data Scientist                                                            7
Postdoctoral Research Fellow                                              7
Senior Data Scientist                                                     6
Campaign Analyst                                                          5
Campaign Project Officer                                                  5
Senior Commercial Analyst                                                 5
Commercial Analyst                                                        5
Java Develop

In [17]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 10 columns):
content            1261 non-null object
industries         1261 non-null object
location           1261 non-null object
title              1261 non-null object
type_empl          1261 non-null object
url                1261 non-null object
wage               1261 non-null int64
suburb             1261 non-null object
experiences        1261 non-null object
title_left_over    1261 non-null object
dtypes: int64(1), object(9)
memory usage: 98.6+ KB


In [18]:
frame.head(50)

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,Over the last 2 years this business has been s...,Information & Communication Technology,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38892320?type=stan...,87575,CBD & Inner Suburbs,General,Data Scientist
1,The Data Architect is responsible for enterpri...,Information & Communication Technology,Brisbane,Data Architect,Full Time,https://www.seek.com.au/job/39015289?type=stan...,114181,CBD & Inner Suburbs,General,
2,About AFSA The Australian Financial Security A...,Information & Communication Technology,Brisbane,Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935314?type=stan...,87747,CBD & Inner Suburbs,General,
3,The Data Engineer (Cloud/Data Lake) will play ...,Information & Communication Technology,Brisbane,Engineers,Full Time,https://www.seek.com.au/job/39015517?type=stan...,114181,CBD & Inner Suburbs,General,Data Engineer - Cloud / Data Lake
4,About AFSA The Australian Financial Security A...,Information & Communication Technology,Brisbane,Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935330?type=stan...,87747,CBD & Inner Suburbs,General,
5,"This is a newly created role, dedicated to for...",Information & Communication Technology,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38890886?type=stan...,150000,CBD & Inner Suburbs,Senior,Senior Data Scientist
6,"​​​​​ About the team Big data, information and...",Consulting & Strategy,Brisbane,Data Analyst,Full Time,https://www.seek.com.au/job/38942934?type=stan...,210000,CBD & Inner Suburbs,General,Director of Big Data
7,The Opportunity Use Your Skills and Make an Im...,Science & Technology,Brisbane,Academic,Full Time,https://www.seek.com.au/job/39040919?type=stan...,102791,Western Suburbs & Ipswich,General,Research Scientist In Field Robotics
8,Department of Transport and Main Roads / Brisb...,Government & Defence,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38871761?type=stan...,88345,CBD & Inner Suburbs,Senior,Senior Data Scientist
9,Senior Analytics Lead - Claims | Cognos | Tabl...,Information & Communication Technology,Brisbane,Analyst,Full Time,https://www.seek.com.au/job/38935642?type=stan...,140000,Brisbane,Junior,"Claims Analytics Lead - Cognos, Tableau, Redsh..."


In [19]:
# experiences = [''] * len(titles)
# left_overs = frame['left_over_title'].values
# new_titles = frame['title'].values

# for key, value in enumerate(left_overs):
# #     print("Value: ", len(value), ' : ' ,value)
#     if len(value) > 1:        
#         is_simple = True
#         # Simple mapping levels
#         for exp in exp_levels:
#             if exp in value:
#                 print("Exp in Exp_levels : ", exp,' ------- ' ,value)
#                 experiences[key] = exp
#                 new_titles[key] = new_titles[key].replace(exp,"").rstrip()
#                 is_simple = False
#                 break
        
#         # Mapping exp base on dictionary
#         if is_simple:
#             for k_dict, v_dict in title_mapping_dictionary.items():
#                 for v_dict_list in v_dict:
#                     if v_dict_list in value:
#                         print("Exp mapping dict : ", k_dict,' ------- ' ,value)
#                         experiences[key] = k_dict
#                         new_titles[key] = new_titles[key].replace(k_dict,"").rstrip()
#                         break
                        
#     else:
#         # Incase not above               
#         experiences[key] = ''
# frame['exp'] = experiences
# frame['title'] = new_titles

In [20]:
# def get_exp(title):
#     for exp in experiences:
#         if exp in title:
#             title = title.replace(exp,"")
#             return title.lstrip().rstrip()
#     return ''
    
# frame['experience'] = frame['title'].map(get_exp)
# frame['experience'].value_counts()